In [6]:
# =============================================================
#              Integração com a API do Sheets 
# =============================================================
from googleapiclient.discovery import build    # Bibliotecas desenvolvidas pelo google 
from google.oauth2 import service_account
from io import StringIO
import requests
import json
from datetime import datetime, timedelta
import pandas as pd

In [7]:
with open('keys_piperun.json', 'r') as file:
    tokens = json.load(file)

access_token_pipe = tokens['access_token']

In [8]:
import json
import pandas as pd 
from datetime import datetime, timedelta

with open('keys_facebook.json', 'r') as file:
    tokens = json.load(file)

access_token = tokens['access_token']
account_id = tokens['account_id']

In [9]:
oportunidades = """26446836
25111597
26071781
26474351"""

oportunidades_n = oportunidades.split("\n")

In [10]:
import requests
import json 
import pandas as pd 

# URL da API do Piperun para obter as notas das oportunidades
api_url_2 = "https://api.pipe.run/v1/deals"
api_url = "https://api.pipe.run/v1/notes"

# Configuração dos cabeçalhos
headers = {
    "accept": "application/json",
    "token": access_token_pipe
}

# Parâmetros da solicitação
params = {
    "deal_id": "26446836"
}

# Faz a solicitação GET para a API do Piperun
response2 = requests.get(api_url_2, headers=headers, params=params)
# Faz a solicitação GET para a API do Piperun
response = requests.get(api_url, headers=headers, params=params)

# Verifica o código de status da resposta
if response2.status_code == 200:
    # A solicitação foi bem-sucedida
    notas_oportunidades = json.loads(response.text)['data']
    df = pd.DataFrame(data=notas_oportunidades)
    df = pd.DataFrame(data=notas_oportunidades)
    # Faça o processamento necessário nas notas das oportunidades aqui
else:
    # Ocorreu um erro na solicitação
    print("Erro na solicitação:", response2.status_code)


In [11]:
import requests
import json
import pandas as pd

# URL da API do Piperun para obter as notas das oportunidades
api_url = "https://api.pipe.run/v1/notes"
api_url_2 = "https://api.pipe.run/v1/deals"

df_final = []

for deal_id in oportunidades_n:

    # Parâmetros da solicitação
    params = {
        "deal_id": deal_id
    }

    # Faz a solicitação GET para a API do Piperun
    response2 = requests.get(api_url_2, headers=headers, params=params)
    response = requests.get(api_url, headers=headers, params=params)

    # Verifica o código de status da resposta
    if response.status_code == 200:
        print(deal_id)
        # A solicitação foi bem-sucedida
        notas_oportunidades = json.loads(response.text)['data']
        oportunidades = json.loads(response2.text)['data']
        df = pd.DataFrame(data=notas_oportunidades)
        df['title'] = oportunidades[0]['title']
        df_final.append(df)
        # Faça o processamento necessário nas notas das oportunidades aqui
    else:
        # Ocorreu um erro na solicitação
        print("Erro na solicitação:", response.status_code)

# Concatenar todos os DataFrames em um único DataFrame final
df_final = pd.concat(df_final, ignore_index=True)


26446836
25111597
26071781
26474351


In [12]:
from bs4 import BeautifulSoup

# Coluna em formato HTML
coluna_html = '<p>Empresa de cadastro:&nbsp;\n\n<!--StartFragment-->Exemplo de texto<!--EndFragment--></p>'

# Converter HTML para texto
soup = BeautifulSoup(coluna_html, 'html.parser')
texto = soup.get_text()

print(texto)


Empresa de cadastro: 

Exemplo de texto


In [13]:
from bs4 import BeautifulSoup

# Função para converter HTML em texto
def html_to_text(html):
    soup = BeautifulSoup(html, 'html.parser')
    texto = soup.get_text()
    return texto

# Aplicar a conversão em toda a coluna 'text'
df_final['text'] = df_final['text'].apply(lambda html: html_to_text(html))


In [14]:
with open('keys_redshift.json', 'r') as file:
    tokens = json.load(file)

host = tokens['host']
port = tokens['port']
database = tokens['database']
user = tokens['user']
password = tokens['password']

In [15]:
import psycopg2

# Query SQL
query = """select * from piperun.deal_customfields dc 
where dc.customfields_id = 156930;"""

# Conexão ao Redshift
conn = psycopg2.connect(
    host=host,
    port=port,
    database=database,
    user=user,
    password=password
)

# Criação de um cursor
cur = conn.cursor()

# Execução da consulta
cur.execute(query)

# Recuperação dos resultados
results = cur.fetchall()
colunas = [desc[0] for desc in cur.description]

# Fechamento do cursor e da conexão
cur.close()
conn.close()


In [16]:

df_redshift = pd.DataFrame(results, columns=colunas)

# Juntar os DataFrames usando o campo "deal_id" como chave de junção
df_merged = pd.merge(df_final, df_redshift, on='deal_id', how='left')

# Imprimir o resultado final
print(df_merged.columns)

Index(['id_x', 'account_id', 'user_id', 'pipeline_id', 'stage_id', 'deal_id',
       'company_id', 'person_id', 'call_id', 'user_name', 'text', 'created_at',
       'updated_at_x', 'title', 'customfields_id', 'type', 'value', 'id_y',
       'updated_at_y'],
      dtype='object')


In [17]:
print("df_redshift",df_redshift.columns,"\ndf_final", df_final.columns)

df_redshift Index(['deal_id', 'customfields_id', 'type', 'value', 'id', 'updated_at'], dtype='object') 
df_final Index(['id', 'account_id', 'user_id', 'pipeline_id', 'stage_id', 'deal_id',
       'company_id', 'person_id', 'call_id', 'user_name', 'text', 'created_at',
       'updated_at', 'title'],
      dtype='object')


In [18]:
# Fazer um left join entre df_final e df_redshift usando o campo "deal_id" como chave de junção
df_merged = pd.merge(df_final, df_redshift[['deal_id', 'value']], on='deal_id', how='left')

# Imprimir o resultado final
df_merged = df_merged.rename({'value': 'id_jira'}, axis = 1)

In [19]:
def sobe_tabela(id, pagina, df_merged):
    SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
    SERVICE_ACCOUNT_FILE = r"keys_google.json"  # Arquivo com as crednciais 

    creds = None
    creds = service_account.Credentials.from_service_account_file(    # Função com a aplicação das credenciais 
            SERVICE_ACCOUNT_FILE, scopes=SCOPES)

    SAMPLE_SPREADSHEET_ID  = id # Pasta marketing_facebook
    service = build('sheets', 'v4', credentials=creds)

    # Call the Sheets API 
    sheet = service.spreadsheets()

    sheet.values().clear(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=f"{pagina}!A:K").execute()
    if "creted_at" in df_merged.columns:
        histories = df_merged.sort_values(by=['created_at'])
    histories = df_merged.sort_values(by=['deal_id'])
    histories.fillna(" ",inplace=True)
    planilha_histories = [histories.columns.tolist()] + histories.values.tolist()

    request = sheet.values().update(spreadsheetId=SAMPLE_SPREADSHEET_ID, 
                                    range=f"{pagina}!A:O",
                                    valueInputOption="USER_ENTERED",
                                    body={"values": planilha_histories}).execute()
    print(request)

In [20]:
sobe_tabela("xxxxxxxxxxxxxxxxxxxxxxxxxx","notas_deals",df_merged)

{'spreadsheetId': 'xxxxxxxxxxxxxxxxxxxxxxxxxx', 'updatedRange': 'notas_deals!A1:O12', 'updatedRows': 12, 'updatedColumns': 15, 'updatedCells': 180}


In [21]:
df_merged.columns

Index(['id', 'account_id', 'user_id', 'pipeline_id', 'stage_id', 'deal_id',
       'company_id', 'person_id', 'call_id', 'user_name', 'text', 'created_at',
       'updated_at', 'title', 'id_jira'],
      dtype='object')

In [22]:
import pandas as pd

# Supondo que você já tenha um DataFrame chamado df com as colunas id_jira e text

# Agrupar por id_jira e juntar os textos
df_grouped = df_merged.groupby('deal_id')['text'].apply(' '.join).reset_index()

In [23]:
sobe_tabela("xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx","notas_deals_agrupada",df_grouped)

{'spreadsheetId': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx', 'updatedRange': 'notas_deals_agrupada!A1:B5', 'updatedRows': 5, 'updatedColumns': 2, 'updatedCells': 10}
